# `df.style`

possible API:

We'd like to conditionally format a dataframe, to style cells according to

- The cell's value (relative to the row / column / table)
- The cell's position (relative to the row / column / table)

To do this, we'll add a new object `Styler`. The constructor for `Styler` is simple

```python
styler = Styler(data)
```

`data` should be a DataFrame (or a Series probably).

We'll encourage method chaining to incrementally build up your styles before translating to HTML and rendering. The interaction is thus:

```python

(s.apply(style_func1)
  .apply(style_func2, arg1=1, arg2=2)
  .apply(style_func3, axis=1)
  .applymap(style_func4)
  .render())

```

We can potentially include some "builtins" that should prove broadly useful, such as `highlight_null`, and `gradient` / `color_background`.

Each style function should take a Series and return a like-indexed Series with strings of `'css property: css value'` (I'm not crazy about using strings here).
`applymap` should be likewise, but it operates on the `DataFrame`.

As an example, here's a style function for highlighting nulls:

```python
def highlight_null(self, null_color='red', notnull_color=None):
    if notnull_color is None:
        notnull_color = ''
    c = (self.data.where(self.data.isnull(),
                         'background-color: %s' % notnull_color)
         .fillna('background-color: %s' % null_color))
    return c 
```

# Implementation bits

We'll keep an internal `ctx` or context object on the `styler` instance. This is a `defaultdict(list)` object mapping either

  - (row_label, column_label): ['property: value']
  - (row_position, column_positoin): ['property: value']

I've implemented it using labels for now, but duplicates would break that.
As new styles are applied, the `ctx` is appended to.

The second step is translating, which involves turning that `ctx` into a dictionary ready to be passed to a Jinja template. Each cell of the DataFrame gets its own HTML id. We'll build up a `<style>` section where the cell `id`s are selectors and the values are the `property: value` pairs from the `ctx`.

This `dict` is dumped into a Jinja template, which upon `render`ing will return the HTML.

Note that this is entirely distinct from the `to_html` codepath. This is probably not ideal, but it made implemenation much easier.


# Background

@yp did a bunch of work on this in https://github.com/pydata/pandas/issues/3190.
I'm using his idea and much of his implementation of using Jinja templates to render translate from the DataFrame with styles to HTML.

# An example

glad you asked.

In [1]:
import seaborn as sns

In [2]:
from pandas.core.style import Styler  # to be added to the pandas namespace

from IPython.display import HTML

np.random.seed(24)
df = pd.DataFrame({'A': np.linspace(1, 10, 10)})
df = pd.concat([df, pd.DataFrame(np.random.randn(10, 4), columns=list('BCDE'))],
               axis=1)
df.iloc[0, 2] = np.nan

In [3]:
cm = sns.light_palette("green", as_cmap=True)

In [4]:
s = df.style.color_bg_range(cmap=cm)
s

In [5]:
def f(x):
    """Series -> foo"""
    return pd.Series(["background-color: red"], index=x.index, name=x.name)

Slicing is supported in `apply` and `applymap` with the subset keyword.
For now you can't reduce the dimensionality with a slice, i.e. do `pd.IndexSlice[:, ['C']]` instead of `pd.IndexSlice[:, 'C']]`. I suspect I'll be able to remove this limitation, but I don't want to much magic atm.

In [6]:
s.apply(f, subset=pd.IndexSlice[:, ['C']])

In [7]:
s.apply(f, axis=1, subset=pd.IndexSlice[[4, 5], :])

We reuse our `display.precision` to control float formatting.
Method chaining is encouraged.

In [10]:
with pd.option_context('display.precision', 2):
    html = (df.style.color_bg_range()
              .set_properties(color='white')
              .text_shadow())
html

A few example styling functions. Perhaps we'll have a repo of these somewhere.

In [11]:
def color_bg_range(s, cmap='PuBu'):
    """Color background in a range according to the data."""
    normed = (s - s.min()) / (s.max() - s.min())
    colors = [rgb2hex(x) for x in plt.cm.get_cmap(cmap)(normed)]
    return pd.Series(['background-color: %s' % color for color in colors], index=s.index)

def color_font_even(s, even_color='black', odd_color='red'):
    """"""
    colors = [even_color if x % 2 else odd_color for x in s]
    return pd.Series(['font-color: %s' % color for color in colors], index=s.index)

def striped(s):
    bg_colors = ["black" if i % 2 else "white" for i in range(len(s))]
    fn_colors = ["white" if i % 2 else "black" for i in range(len(s))]
    return pd.Series(['background-color: %s; color: %s' % (bg, c)
                      for bg, c in zip(bg_colors, fn_colors)], index=s.index)

def checker(table):
    nrow, ncol = table.shape
    colors = [['background-color: %red' if r % 2 or c % 2 else 'background-color: black'
               for r in range(ncol)]
              for c in range(nrow)]
    return pd.DataFrame(colors, index=table.index, columns=table.columns)


In [12]:
df.style.apply(striped)

In [13]:
df.style.apply(striped, axis=1)

Fun stuff.

In [14]:
def cylon(s):
    tpl = """
  background-color: red;
  background-image: -webkit-linear-gradient(    left, rgba( 0,0,0,0.9 ) 25%, rgba( 0,0,0,0.1 ) 50%, rgba( 0,0,0,0.9 ) 75%);
  background-image:    -moz-linear-gradient(    left, rgba( 0,0,0,0.9 ) 25%, rgba( 0,0,0,0.1 ) 50%, rgba( 0,0,0,0.9 ) 75%);
  background-image:      -o-linear-gradient(    left, rgba( 0,0,0,0.9 ) 25%, rgba( 0,0,0,0.1 ) 50%, rgba( 0,0,0,0.9 ) 75%);
  background-image:         linear-gradient(to right, rgba( 0,0,0,0.9 ) 25%, rgba( 0,0,0,0.1 ) 50%, rgba( 0,0,0,0.9 ) 75%);
  color: white;
  height: 100%;
  width: 20"""
    return pd.Series([tpl for i in s], index=s.index, name=s.name)

In [15]:
df.style.apply(cylon)

In [ ]:
df.style.`

In [20]:
from IPython.html import widgets
@widgets.interact
def f(h_neg=(0, 359, 1), h_pos=(0, 359), s=(0., 99.9), l=(0., 99.9)):
    return (df
              .style
              .color_bg_range(cmap=sns.palettes.diverging_palette(h_neg=h_neg, 
                                                                  h_pos=h_pos, 
                                                                  s=s, 
                                                                  l=l,
                                                                  as_cmap=True))
              .highlight_null()
    )